# Installing unavailable libraries:

In [ ]:
! pip install selenium
! pip install requests
! pip install schedule

### Note: In addition to these libraries, you'll also need to download ChromDriver from the internet with the version compatible with your Chrome Browser.

# Importing the required Libraries:

In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time, schedule, requests, json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup

# Setting up the Chrome Webdriver:

In [ ]:
options = Options()
options.add_argument('--headless') # Runs in Headless Mode
service = Service('C:\Program Files\Google\Chrome\Application\chromedriver.exe')
driver = wd.Chrome(service = service, options = options)

# Telegram credentials (User's Input Required):

In [ ]:
# Replace the missing values with the ones specific to your case
telegram_chat_id = "-XXXXXXXXXXXXXX"
telegram_bot_id = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

# Developing Monitor for Heimstaden:

In [ ]:
def check_Heimstaden():
        
    """Heimstaden"""
    try:

        print("\n# Heimstaden #\n")
        
        # Defining the list of target locations you are interested in
        target_locations = ["Linköping - Lambohov", "Linköping - Ryd","Linköping - Vallastaden","Linköping - Vasastan","Linköping - Centrum","Linköping - Gottfridsberg"]

        # List to store results
        results = []
        
        URL = "https://heimstaden.com/se/sok-lagenhet/?text=Link%C3%B6ping%2C%20Sweden&display_mode=picture&search_version=1.5&order=&tracking_object_type=Apartment&google_place_id=ChIJlZ8EmnFuWUYRcI7kQfP-AAQ&google_place_type=locality&google_place_name=Link%C3%B6ping&number_of_rooms_min=1&number_of_rooms_max=5&rent_min=0&rent_max=16000&size_min=0&size_max=300&properties_true_false%5B%5D=not_student&offset=15"
        driver.get(URL)
        
        # Waiting for the parent containers to be visible
        parent_containers = WebDriverWait(driver, 15).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, 'object-teaser-picture-card'))
        )

        # Looping through each parent container
        for container in parent_containers:
            # Find the <h1> tag within the current container for location
            h1_element = container.find_element(By.CLASS_NAME, 'object-teaser-picture-card__content-location')
            location_text = h1_element.text.strip()

            # Check if the location matches one of the target locations
            if location_text in target_locations:
                
                # Extracting the name of the Offer
                h_element = container.find_element(By.CSS_SELECTOR, 'h2.object-teaser-picture-card__content-heading')
                offer_name = h_element.text.strip()
                
                # Extracting the URL of the Offer
                a_element = container.find_element(By.CSS_SELECTOR, 'a.main-img')
                href_url = a_element.get_attribute('href')

                # Storing the matched location and corresponding Offer name and URL
                results.append({'name': offer_name, 'location': location_text.split(' - ')[1], 'url': href_url})
                
            else:
                pass
            

        print("Offers of interest: ", len(results), '\n')
        #print(results)
        
        # Output the results
        for result in results:
            print(f"{results.index(result)+1}) {result['name']}, {result['location']}, URL: {result['url']}")
            
        
        # Deputing Telegram bot to notify based on availability of accommodation:
        
        message_text = ""

        for result in results:
            message_text += f"{results.index(result)+1}) {result['name']}, {result['location']}\nURL: {result['url']}\n\n"

        url = "https://api.telegram.org/bot" + telegram_bot_id + "/sendMessage"
        data = {
            "chat_id": telegram_chat_id,
            "text": f"*Heimstaden*\n\nOffers of interest: {len(results)}\n\n{message_text}",
            "parse_mode": "Markdown"
                }
        try:
            response = requests.request(
                "POST",
                url,
                params=data
            )
            telegram_data = json.loads(response.text)

            if telegram_data["ok"] == True:
                print("Notification Raised.\n")                   
                

        except Exception as e:
            print("An error occurred in sending the Notification via Telegram")
            print(e)

    except Exception as e:
        print('Error: ', e)

# Developing Monitor for Stångåstaden:

In [ ]:
def check_Stångåstaden():
        
    try:
        
        print(f"\n# Stångåstaden #\n")

        # Navigating to the Website with the filters set to 2 RUM, 3 RUM, 4 RUM for the set locations
        
        URL1 = 'https://www.stangastaden.se/sokledigt/ledigabostader/?actionId=&omraden%5B%5D=INNER&omraden%5B%5D=LAMBO&omraden%5B%5D=RYD&omraden%5B%5D=T1VAFR&omraden%5B%5D=VASA&omraden%5B%5D=ABYGOT&oboTyper=A2&ytaMin=&ytaMax=&hyraMax='
        URL2 = 'https://www.stangastaden.se/sokledigt/ledigabostader/?actionId=&omraden%5B%5D=INNER&omraden%5B%5D=LAMBO&omraden%5B%5D=RYD&omraden%5B%5D=T1VAFR&omraden%5B%5D=VASA&omraden%5B%5D=ABYGOT&oboTyper=A3&ytaMin=&ytaMax=&hyraMax='
        URL3 = 'https://www.stangastaden.se/sokledigt/ledigabostader/?actionId=&omraden%5B%5D=INNER&omraden%5B%5D=LAMBO&omraden%5B%5D=RYD&omraden%5B%5D=T1VAFR&omraden%5B%5D=VASA&omraden%5B%5D=ABYGOT&oboTyper=A4&ytaMin=&ytaMax=&hyraMax='
        
        URLS = (URL1, URL2, URL3)
        
        for URL in URLS:
            print(f"Results for {URLS.index(URL)+2} rooms:\n")
            driver.get(URL)

            # Locating the element and retrieving the text

            element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="main-content"]/div/div/div/div[2]/div[2]/div'))
            )
            number = element.text.split()[5]
            print("Offers available: ", number)

            link_elements = driver.find_elements(By.CSS_SELECTOR, 'a.apartment-item')
            
            urls = []
            results = []
            
            for element in link_elements:
                url = element.get_attribute('href')
                urls.append(url)

            for offer in urls:
                driver.get(offer)
                
                # Waiting for the <div> with class 'adress mb-2 mt-0 lg:mt-10' to be visible to fetch the Offer's name
                offer_name = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.adress.mb-2.mt-0.lg\:mt-10'))
                )
                offer_name = offer_name.text.strip()
                
                # Fetching the location of the offer
                div_element = driver.find_element(By.CLASS_NAME, 'icon-pin')
                location = div_element.text
                
                print(f"\n{urls.index(offer)+1}) {offer_name}, {location}")
                
                
                # Locating the element with class 'IntresseMeddelande'
                element = driver.find_element(By.CSS_SELECTOR, 'div.IntresseMeddelande')

                # Extracting text from the element for last date of application
                dt = element.text.split()[3]
                print(f"Last date: {dt}")
                

                dt1 = datetime.strptime(dt, "%Y-%m-%d")

                today = datetime.now().date()

                if dt1.date() <= today + timedelta(days = 3):
                    points = driver.find_element(By.CSS_SELECTOR, 'div[data-widget="objektintressestatus"]')
                    html_content = points.get_attribute('innerHTML')


                    # Parseing the HTML content using BeautifulSoup
                    soup = BeautifulSoup(html_content, 'html.parser')

                    # Finding all <br> tags
                    br_tags = soup.find_all('br')

                    # Extracting and cleaning the numbers following each <br>
                    numbers = []
                    try:
                        for br in br_tags:
                            # Extracting the text node immediately following each <br> tag
                            next_node = br.find_next_sibling(text=True)
                            if next_node:
                                # Cleaning the text
                                cleaned_text = next_node.strip().replace('\xa0', '')
                                if cleaned_text:
                                    # Appending only if it matches the expected pattern (numbers with optional text)
                                    if cleaned_text[0].isdigit():
                                        numbers.append(int(cleaned_text[3:]))                                     
                    
                    except Exception as e:
                        print(e)

                    # Printing the first three numbers
                    numbers = numbers[0:-2]
                    print(f"Top 3 points: {numbers}")
                    print(f'URL: {offer}')

                else:
                    numbers = "The offer isn't within 3 days"
                    print("The offer isn't within 3 days")
                    print(f'URL: {offer}')
                    
                results.append({'name': offer_name, 'location': location, 'last_date': dt, 'top_3': numbers, 'url': offer})

            message_text = ""
                          
            for result in results:
                if isinstance(result['top_3'], list):
                    Annexure = "Top 3 points: "
                else:
                    Annexure = ""
                    
                message_text += f"{results.index(result)+1}) {result['name']}, {result['location']}\nLast Date: {result['last_date']}\n{Annexure}{result['top_3']}\nURL: {result['url']}\n\n"

            url = "https://api.telegram.org/bot" + telegram_bot_id + "/sendMessage"
            data = {
                "chat_id": telegram_chat_id,
                "text": f"*Stångåstaden*\n\nResults for {URLS.index(URL)+2} rooms:\nOffers of interest: {len(results)}\n\n{message_text}",
                "parse_mode": "Markdown"
                    }
            try:
                response = requests.request(
                    "POST",
                    url,
                    params=data
                )
                telegram_data = json.loads(response.text)

                if telegram_data["ok"] == True:
                    print("\nNotification Raised.\n")                   


            except Exception as e:
                print("An error occurred in sending the Notification via Telegram")
                print(e)
            
    except Exception as e:
            print('Error: ', e)

# Developing Monitor for Studentbostäder:

In [ ]:
def check_Studentbostäder():  
    
    try:
        
        print(f"\n# Studentbostäder #\n")

        # Navigating to the Website with the filters set to 2 RUM, 3 RUM, 5 RUM for the set locations
        
        URL1 = 'https://www.studentbostader.se/soker-bostad/lediga-bostader/?actionId=&omraden=&egenskaper=&oboTyper=SA2'
        URL2 = 'https://www.studentbostader.se/soker-bostad/lediga-bostader/?actionId=&omraden=&egenskaper=&oboTyper=SA3'
        URL3 = 'https://www.studentbostader.se/soker-bostad/lediga-bostader/?actionId=&omraden=&egenskaper=&oboTyper=SA5'
        
        URLS = (URL1, URL2, URL3)
        
        for URL in URLS:
            if URLS.index(URL) < 2:
                print(f"Results for {URLS.index(URL)+2} rooms:\n")
                driver.get(URL)
            else:
                print(f"Results for {URLS.index(URL)+3} rooms:\n")
                driver.get(URL)

            # Locating the element and retrieving the text

            element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="main-content"]/div/div/div[2]/div/div/div[2]/div[1]/div[1]/div'))
            )
            n_offers = element.text.split()[3]
            print("Offers available: ", n_offers)

            link_elements = driver.find_elements(By.CSS_SELECTOR,"a.my-4.lg\\:my-2")
            
            urls = []
            results = []
            
            for element in link_elements:
                url = element.get_attribute('href')
                urls.append(url)                
            
            for offer in urls:
                driver.get(offer)
                
                # Waiting for the <h1> with class 'adress' to be visible to fetch the Offer's name
                offer_name = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.adress'))
                )
                offer_name = offer_name.text.strip()
                
                # Fetching the location of the offer
                location_element = driver.find_element(By.CSS_SELECTOR, 'p.inline-block.float-right.mb-0')
                location = location_element.text
                
                print(f"\n{urls.index(offer)+1}) {offer_name}, {location}")
                
                
                # Locating the element with class 'IntresseMeddelande'
                element = driver.find_element(By.CSS_SELECTOR, 'div.IntresseMeddelande')

                # Extracting text from the element for last date of application
                dt = element.text.split()[3]
                print(f"Last date: {dt}")
                

                points = driver.find_element(By.CSS_SELECTOR, 'div[data-widget="objektintressestatus"]')
                html_content = points.get_attribute('innerHTML')


                # Parseing the HTML content using BeautifulSoup
                soup = BeautifulSoup(html_content, 'html.parser')

                # Finding all <br> tags
                br_tags = soup.find_all('br')

                # Extracting and cleaning the numbers following each <br>
                numbers = []
                try:
                    for br in br_tags:
                        # Extracting the text node immediately following each <br> tag
                        next_node = br.find_next_sibling(text=True)
                        if next_node:
                            # Cleaning the text
                            cleaned_text = next_node.strip().replace('\xa0', '')
                            if cleaned_text:
                                # Appending only if it matches the expected pattern (numbers with optional text)
                                if cleaned_text[0].isdigit():
                                    numbers.append(int(cleaned_text[3:]))                                     
                    
                except Exception as e:
                    print(e)

                # Printing the first three numbers
                numbers = numbers[0:-2]
                print(f"Top 3 points: {numbers}")
                print(f'URL: {offer}')
                    
                results.append({'name': offer_name, 'location': location, 'last_date': dt, 'top_3': numbers, 'url': offer})

            message_text = ""
                          
            for result in results:
                Annexure = "Top 3 points: "
                    
                message_text += f"{results.index(result)+1}) {result['name']}, {result['location']}\nLast Date: {result['last_date']}\n{Annexure}{result['top_3']}\nURL: {result['url']}\n\n"

            url = "https://api.telegram.org/bot" + telegram_bot_id + "/sendMessage"
            data = {
                "chat_id": telegram_chat_id,
                "text": f"*Studentbostäder*\n\nResults for {URLS.index(URL)+2} rooms:\nOffers available: {n_offers}\n\n{message_text}",
                "parse_mode": "Markdown"
                    }
            try:
                response = requests.request(
                    "POST",
                    url,
                    params=data
                )
                telegram_data = json.loads(response.text)

                if telegram_data["ok"] == True:
                    print("\nNotification Raised.\n")                   


            except Exception as e:
                print("An error occurred in sending the Notification via Telegram")
                print(e)
            
    except Exception as e:
            print('Error: ', e)

# Automating the process with a Scheduler:

In [ ]:
# Schedule the task to run every minute

schedule.every().day.at("11:00").do(check_Stångåstaden)
schedule.every().day.at("10:15").do(check_Heimstaden)
schedule.every().day.at("17:00").do(check_Heimstaden)
schedule.every().thursday.at("11:00").do(check_Studentbostäder)
schedule.every().sunday.at("11:00").do(check_Studentbostäder)

# Run the scheduler
while True:
    try:
        schedule.run_pending()
        time.sleep(30)
        
    except Exception as e:
        print(e)